# Exception Basics

**Chapter 6 - Learning Python, 5th Edition**

Python's exception handling mechanism provides structured error management through
`try/except/else/finally` blocks. Understanding the exception hierarchy and control
flow is essential for writing robust, maintainable code.

## The Exception Hierarchy

All exceptions inherit from `BaseException`. The common ones we catch inherit from `Exception`.

```
BaseException
├── SystemExit
├── KeyboardInterrupt
├── GeneratorExit
└── Exception
    ├── StopIteration
    ├── ArithmeticError
    │   ├── ZeroDivisionError
    │   └── OverflowError
    ├── LookupError
    │   ├── IndexError
    │   └── KeyError
    ├── TypeError
    ├── ValueError
    ├── AttributeError
    ├── OSError
    │   ├── FileNotFoundError
    │   └── PermissionError
    └── ...
```

In [ ]:
# Inspecting the exception hierarchy
print(f"ZeroDivisionError MRO: {ZeroDivisionError.__mro__}")
print(f"FileNotFoundError MRO: {FileNotFoundError.__mro__}")
print(f"KeyError MRO: {KeyError.__mro__}")

## try / except / else / finally

The full exception handling syntax with all four clauses:

In [ ]:
def divide(a: float, b: float) -> float | None:
    """Demonstrate full try/except/else/finally control flow."""
    try:
        result = a / b
    except ZeroDivisionError:
        print("Error: Division by zero")
        return None
    except TypeError as e:
        print(f"Error: Invalid types - {e}")
        return None
    else:
        # Only runs if NO exception was raised
        print(f"Success: {a} / {b} = {result}")
        return result
    finally:
        # ALWAYS runs, even if there's a return
        print("Cleanup: division operation complete")


print("--- Normal case ---")
divide(10, 3)

print("\n--- Error case ---")
divide(10, 0)

print("\n--- Type error case ---")
divide(10, "abc")

## Catching Multiple Exceptions

You can catch multiple exception types in a single handler or chain handlers:

In [ ]:
def safe_access(data: dict | list, key: str | int) -> str:
    """Safely access data structures with descriptive errors."""
    try:
        return data[key]
    except (KeyError, IndexError) as e:
        # Catch multiple exceptions in one handler
        return f"Access error: {type(e).__name__}: {e}"
    except TypeError as e:
        return f"Type mismatch: {e}"


print(safe_access({"a": 1}, "b"))      # KeyError
print(safe_access([1, 2, 3], 10))       # IndexError
print(safe_access({"a": 1}, [1, 2]))    # TypeError (unhashable)

## The `as` Clause and Exception Attributes

The `as` keyword captures the exception object for inspection:

In [ ]:
try:
    int("not_a_number")
except ValueError as e:
    print(f"Exception type: {type(e).__name__}")
    print(f"Exception args: {e.args}")
    print(f"String representation: {e}")
    print(f"Is it an Exception? {isinstance(e, Exception)}")

## EAFP vs LBYL

Two contrasting error-handling philosophies:

- **EAFP** (Easier to Ask Forgiveness than Permission): Try the operation, catch exceptions
- **LBYL** (Look Before You Leap): Check conditions before attempting the operation

Python idiom strongly favors EAFP.

In [ ]:
config: dict[str, str] = {"host": "localhost", "port": "8080"}


# LBYL style (non-Pythonic)
def get_port_lbyl(cfg: dict[str, str]) -> int:
    if "port" in cfg:
        value = cfg["port"]
        if isinstance(value, str) and value.isdigit():
            return int(value)
    return 80  # default


# EAFP style (Pythonic)
def get_port_eafp(cfg: dict[str, str]) -> int:
    try:
        return int(cfg["port"])
    except (KeyError, ValueError):
        return 80  # default


print(f"LBYL: {get_port_lbyl(config)}")
print(f"EAFP: {get_port_eafp(config)}")
print(f"Missing key EAFP: {get_port_eafp({})}")

## Re-raising Exceptions

Use bare `raise` to re-raise the current exception after logging or partial handling:

In [ ]:
import logging

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)


def process_data(data: list[int]) -> float:
    """Process data with logging on failure."""
    try:
        return sum(data) / len(data)
    except ZeroDivisionError:
        logger.warning("Attempted to process empty data")
        raise  # Re-raise the original exception


try:
    process_data([])
except ZeroDivisionError as e:
    print(f"Caught re-raised exception: {e}")